In [ ]:
# General support
import math
import tensorflow as tf
import numpy as np
import json

# General lucid code
import lucid.modelzoo.vision_models as models
from lucid.modelzoo.vision_models import Model

In [ ]:
options = {
    'model': 'Dd_m2_3_gpu_tf_pdtest',
    'split': 'train'
}

In [ ]:
model = Model.load("/Users/longm/Desktop/Dd_m2_3_gpu_tf_pbtest.pb")
model.load_graphdef()

In [ ]:
sess = tf.InteractiveSession()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
train_dir = '/Users/longm/Desktop/train'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (256,256), batch_size = 25, class_mode = 'categorical')

In [ ]:
(x_train, y_train) = train_generator.next()

In [ ]:
image_batch = tf.reshape(x_train, [25,256,256,3])

In [ ]:
labels_batch = tf.reshape(y_train, [25,5])

In [ ]:
from lucid.optvis import render

In [ ]:
T = render.import_model(model, image_batch, image_batch)

In [ ]:
def save_data(base_dir, options, activations, attributions):

  # spatial activations
  activations_path = optionsToURL(base_dir + "activations", "npy", options)
  with gfile.GFile(activations_path, "w") as f:
    np.save(f, activations)

  # spatial attributions to final classes
  attributions_path = optionsToURL(base_dir + "attribution", "npy", options)
  with gfile.GFile(attributions_path, "w") as f:
    np.save(f, attributions)


In [ ]:
def fwd_gradients(ys, xs, d_xs):
  
  v = tf.zeros_like(ys)
  g = tf.gradients(ys, xs, grad_ys=v)
  return tf.gradients(g, v, grad_ys=d_xs)

In [ ]:
number_of_images = int(25)
options['sample_images'] = number_of_images
number_of_pages = 1
number_of_images_per_page = number_of_images / number_of_pages

layers = [
  "Conv2D",
  "Conv2D_1", "Conv2D_2", "Conv2D_3","Conv2D_4", 
  "Conv2D_5", "Conv2D_6", "Conv2D_7", "Conv2D_8", "Conv2D_9", "Conv2D_10", "Conv2D_11", "Conv2D_12"
]

for layer_name in reversed(layers):
  print()
  print(layer_name)
  options['layer'] = layer_name

  d_previous = tf.placeholder("float32")
  d_logit = fwd_gradients(T("output"), T(layer_name), d_previous)[0]

  
  zeros = None
  print(number_of_pages)
  
  for p in range(int(1)):
    activations = []
    attributions = []
  
    for n in range(int(25)):

      # evaluate
#      vec, image, label = sess.run([T(layer_name), x_train, y_train])
      vec = sess.run([T(layer_name)])
      image_other = sess.run([image_batch])
      label = sess.run([labels_batch])

      # sample one random position in the image, minus the edges
      options['sample_type'] = 'random'
      n_x = np.random.randint(1, vec.shape[1])
      n_y = np.random.randint(1, vec.shape[2])
      
      # Compute logit attribution
      if zeros is None:
        zeros = np.zeros(vec.shape)
      else:
        zeros[:] = 0
      zeros[0, n_x, n_y] = vec[0, n_x, n_y]
      logit_attr = d_logit.eval({T(layer_name): vec, d_previous: zeros})

      # top attributions for spatial activation:
      top_attribution_class_index = int(np.argsort(-logit_attr[0])[0])
      top_attribution_class_label = model.labels[top_attribution_class_index]

      activations.append(vec[0, n_x, n_y])
      attributions.append(logit_attr[0])

    # progress indicator
    print(p + 1)

    # save files to bigstore
    options['page'] = '{}_of_{}'.format(p + 1, number_of_pages)
#    save_data(base_dir_gcs + layer_name + "/", options, activations, attributions)
    save_data('/Users/longm/Desktop/collect_activations_model1/' + layer_name + '/', options, activations, attributions)